# Sparse COO

> The most efficient S-representation is the COO representation. It's however also the least user friendly:

In [ ]:
# hide
import jax.numpy as jnp

import sax

In [ ]:
def my_coo():
    num_input_ports = 4
    num_output_ports = 1
    pm = {
        **{f"in{i}": i for i in range(num_input_ports)},
        **{f"out{i}": i + num_input_ports for i in range(num_output_ports)},
    }
    thru = jnp.ones(num_input_ports)
    i = jnp.arange(0, num_input_ports, 1)
    j = jnp.zeros_like(i) + num_input_ports

    # make reciprocal
    i, j = jnp.concatenate([i, j]), jnp.concatenate([j, i])
    thru = jnp.concatenate([thru, thru], 0)
    return (i, j, thru, pm)

In [ ]:
circuit, _ = sax.circuit(
    netlist={
        "instances": {
            "coo": "coo",
        },
        "connections": {},
        "ports": {
            "in0": "coo,in0",
            "out0": "coo,out0",
        },
    },
    models={
        "coo": my_coo,
    },
    backend="klu",
    return_type="scoo",
)

In [ ]:
circuit()

In [ ]:
sax.sdict(circuit())